In [307]:
# imports
import os
import io
import re
# import tokenize
# import json
import numpy as np
import pandas as pd

In [308]:
# paths
data_path =  '/home/ourownstory/Documents/SOL/data/'
# zeppelin_folder = '/home/ourownstory/Documents/SOL/data/Zeppelin/token/'
# os.listdir(data_path)
# os.listdir(zeppelin_folder)
out_path = '/home/ourownstory/Documents/SOL/derived/'

In [309]:
df_files_dict = {
    'root': [],
    'file_name': [],
}
for root, subdirs, df_files in os.walk(data_path):
    for file_name in df_files:
        df_files_dict['root'].append(root[len(data_path):])
        df_files_dict['file_name'].append(file_name)
        
df_files = pd.DataFrame.from_dict(df_files_dict)
root_list = df_files['root'].values
root_list = [root.split('/') for root in root_list]
df_files['class'] = [r.pop(0) for r in root_list]
df_files['company'] = [r.pop(0) for r in root_list]
df_files['root'] = ["/".join(r) for r in root_list]
file_name_list = df_files['file_name'].values
df_files['extension'] = [e.split('.')[-1] for e in file_name_list]

# filter all files that are not .sol
df_files = df_files[df_files.pop('extension') == 'sol']

df_files.to_csv(os.path.join(out_path, 'df_files.csv'))


In [310]:
unique_counts = {}
for col in df_files:
    col_counts = pd.Series.value_counts(df_files[col], sort=True)
    unique_counts[col] = col_counts.to_dict()
    col_counts.to_csv(os.path.join(out_path, 'counts_{}.csv'.format(col)))
    
# TODO: make plot ICO vs notICO

In [311]:
# read in contents of files as string
def read_src_as_string(file_name):
    with open(file_name, 'r') as f:
        return f.read()
        
def get_filename_for_row(row):
    filename = os.path.join(
        data_path, 
        row.loc['class'], 
        row.loc['company'], 
        row.loc['root'], 
        row.loc['file_name']
    )
    return filename

def get_file_src_as_string(row):
    return read_src_as_string(get_filename_for_row(row))
    

In [312]:
# execute src reading
df_files['src'] = df_files.apply(get_file_src_as_string, axis=1)

In [313]:
# retrieve imports
def match_imports_from_src_string(src_string):
    regex = "import .*;\n"
    matches = re.findall(regex, src_string)
    matches_cleaned = [m[len("import '"):-len("';\n")] for m in matches]
    return matches_cleaned

def exctract_imports(src_string):
    matches_cleaned = match_imports_from_src_string(src_string)
    imports =  [m.split('/')[-1] for m in matches_cleaned]
    return imports

def exctract_imports_path(src_string):
    matches_cleaned = match_imports_from_src_string(src_string)
    imports_path =  ["/".join(m.split('/')[:-1]) for m in matches_cleaned]
    return imports_path

In [314]:
# execute
df_files['imports'] = df_files['src'].apply(exctract_imports)
df_files['imports_path'] = df_files['src'].apply(exctract_imports_path)

In [315]:
# check files with (many) imports
files_with_imports = df_files[df_files['imports'].apply(lambda x: len(x) > 5)]
len(files_with_imports)
# files_with_imports

# TODO: make plot ICO vs notICO

38

In [316]:
# files_with_imports['imports'].values[0]

In [317]:
# we want to combine imported files with imports
# first check if file_names are unique in a company
# A: no. but only AugurCore and DemeterLife
identical_filenames = df_files.groupby(['company', 'file_name']).size()
identical_filenames[identical_filenames > 1]

company      file_name        
AugurCore    BasicToken.sol       2
             ERC20.sol            2
             ERC20Basic.sol       2
             Initializable.sol    2
             Ownable.sol          2
             StandardToken.sol    2
DemeterLife  Migrations.sol       2
             RefundVault.sol      2
dtype: int64

In [318]:
# look at example duplicate name
augurcore = df_files[df_files['company'] == 'AugurCore']
augurcore[augurcore['file_name'] == 'BasicToken.sol']

,file_name,root,class,company,src,imports,imports_path
124,BasicToken.sol,legacy_reputation,notICO,AugurCore,pragma solidity 0.4.18;\n\n\nimport 'legacy_re...,"[ERC20Basic.sol, SafeMath.sol]","[legacy_reputation, legacy_reputation]"
148,BasicToken.sol,libraries/token,notICO,AugurCore,pragma solidity 0.4.18;\n\n\nimport 'libraries...,"[ERC20Basic.sol, SafeMathUint256.sol]","[libraries/token, libraries/math]"


In [319]:
# compute frequency of imported files
imports_flat = [item for imports in df_files['imports'].values for item in imports]
col_counts = pd.Series(imports_flat).value_counts(sort=True)
col_counts.to_csv(os.path.join(out_path, 'counts_{}.csv'.format('imports')))

# TODO: make plot ICO vs notICO

In [320]:
# save without src to csv
df_files_out = df_files.copy(deep=True)
df_files_out['src'] = df_files['src'].apply(lambda x: len(x))
df_files_out.to_csv(os.path.join(out_path, 'df_files.csv'))


In [401]:
# match imported files with files in dataset
df_files['ID'] = df_files.index.values
df_files['imports_idx'] = None
df_files['imports_zeppelin'] = False
df_files['contains_zeppelin'] = False

verbose = False

for idx in df_files.index.values:
    f = df_files.loc[idx]
    df_files.loc[idx, 'contains_zeppelin'] = 'zeppelin' in f.loc['src'].lower()
    company = f.loc['company']
    files_company = df_files[df_files['company'] == company]
    f_imports = f.loc['imports']
    f_imports_path = f.loc['imports_path']
#     if len(f_imports) < 1:
#         continue
    imports_idx_list = []
    for import_file_name, import_file_path in zip(f_imports, f_imports_path):
        matching_files = files_company[files_company['file_name'] == import_file_name]
        
        if len(matching_files) == 1:
            imports_idx = matching_files.index.values[0]
            
        elif len(matching_files) > 1:
            # handle ties
            target_set = set(import_file_path.split('/'))
            num_joint_roots = []
            for match_root in matching_files['root'].values:
                joint_roots = [1 for r in match_root.split('/') if r in target_set]
                num_joint_roots.append(sum(joint_roots))
            max_matches = max(num_joint_roots)
            if sum(max_matches == m for m in num_joint_roots) > 1:
                # handle tie-tie
                root_len_diff = [abs(len(import_file_path.split('/')) - len(r.split('/'))) 
                                 for r in matching_files['root'].values]
                better_match = root_len_diff.index(min(root_len_diff)) 
            else:
                better_match = num_joint_roots.index(max_matches)      
            imports_idx = matching_files.index.values[better_match]
            
            if verbose:
                print "import root: {}; matching roots: {}".format(import_file_path, matching_files['root'].values)
                print import_file_path in matching_files['root'].values
            
        elif len(matching_files) < 1:
            # not found
            if 'zeppelin' in import_file_path.lower():
                imports_idx = -2
                df_files.at[idx, 'imports_zeppelin'] = True
            else: 
                imports_idx = -1
                if verbose:
                    print "no import match for: ", import_file_name, import_file_path
        imports_idx_list.append(imports_idx)
    
    df_files.at[idx, 'imports_idx'] = imports_idx_list


In [402]:
sum(df_files['imports_zeppelin'] == True)

68

In [403]:
sum(df_files['contains_zeppelin'] == True)

93

In [404]:
imports_z = df_files.groupby(['class', 'company'])['imports_zeppelin'].sum()
contains_z = df_files.groupby(['class', 'company'])['contains_zeppelin'].sum()
z_presence = pd.DataFrame(pd.Series(imports_z))
z_presence['contains_zeppelin'] = contains_z
z_presence

imports_zeppelin  contains_zeppelin
class    company                                           
ICO      Aragon                      3.0                3.0
         AuthenticID                 2.0                2.0
         BATToken                    0.0                0.0
         BitClave                    7.0                7.0
         BitDegree                   1.0                1.0
         Bread                       0.0                6.0
         BullToken                   8.0                9.0
         DemeterLife                 0.0                7.0
         Gameflip                    3.0                3.0
         Hoqu                        3.0                3.0
         Jincor                      9.0                9.0
         Kryptopy                    2.0                2.0
         LordMancer                  5.0                5.0
         Lynyr                       0.0                2.0
         Monetha                     0.0                0.0
         Mysterium                   8.0                9.0
         NapoleonX                   0.0                0.0
         Realisto                    0.0                1.0
         RestartEnergy               5.0                5.0
         SirinLabs                   0.0                2.0
         Status                      0.0                0.0
         Storj                       5.0                5.0
         Vechain                     0.0                0.0
Zeppelin Zeppelin                    0.0                2.0
notICO   AceBusters                  0.0                1.0
         AragonCore                  0.0                2.0
         AugurCore                   0.0                0.0
         EthLance                    0.0                0.0
         EthLottery                  0.0                0.0
         GnosisMarkets               0.0                0.0
         MonethaTrust                7.0                7.0
         Solidified                  0.0                0.0
         StakeTree                   0.0                0.0

In [412]:
# check if is imported 
# -> not imported files will be combined with the files they import

imported_idxs = set([item for imports in df_files['imports_idx'].values for item in imports])
df_files['is_imported'] = df_files['ID'].apply(lambda x: x in imported_idxs)

In [413]:
df_files

,file_name,root,class,company,src,imports,imports_path,imports_zeppelin,contains_zeppelin,imports_idx,ID,is_imported
0,PullPayInterface.sol,,notICO,AceBusters,pragma solidity ^0.4.11;\n\ncontract PullPayIn...,[],[],False,False,[],0,True
1,ERC20Basic.sol,,notICO,AceBusters,pragma solidity ^0.4.11;\n\n\n/*\n * ERC20Basi...,[],[],False,False,[],1,True
2,Migrations.sol,,notICO,AceBusters,pragma solidity ^0.4.11;\n\ncontract Migration...,[],[],False,False,[],2,False
3,ERC223Basic.sol,,notICO,AceBusters,pragma solidity ^0.4.11;\n\nimport './ERC20Bas...,[ERC20Basic.sol],[.],False,False,[1],3,True
4,ERC20.sol,,notICO,AceBusters,pragma solidity ^0.4.11;\n\n\nimport './ERC223...,[ERC223Basic.sol],[.],False,False,[3],4,True
5,ERC223ReceivingContract.sol,,notICO,AceBusters,pragma solidity ^0.4.11;\n\n /*\n * Contract t...,[],[],False,False,[],5,True
6,SafeMath.sol,,notICO,AceBusters,pragma solidity ^0.4.11;\n\n\n/**\n * @title S...,[],[],False,False,[],6,True
7,PullPayment.sol,satelites,notICO,AceBusters,pragma solidity ^0.4.11;\n\n\nimport '../SafeM...,"[SafeMath.sol, Ownable.sol, ControllerInterfac...","[.., ../ownership, ../controller]",False,False,"[6, 22, 15]",7,True
8,Storage.sol,satelites,notICO,AceBusters,"pragma solidity ^0.4.11;\n\nimport ""../ownersh...",[Ownable.sol],[../ownership],False,False,[22],8,True
9,Nutz.sol,satelites,notICO,AceBusters,"pragma solidity ^0.4.11;\n\nimport ""../ERC20.s...","[ERC20.sol, Ownable.sol, ControllerInterface.s...","[.., ../ownership, ../controller, .., ..]",False,True,"[4, 22, 15, 5, 0]",9,True
